## Setting Up

In [ ]:
import tensorflow as tf 
from tensorflow import keras 
from keras import layers,models,regularizers
from keras.preprocessing import image 
from keras.preprocessing.image import ImageDataGenerator 
from keras.callbacks import ModelCheckpoint,EarlyStopping 
import numpy as np 
import pandas as pd 
import os


## Creating Model

In [ ]:
model = models.Sequential()

model.add(layers.Conv2D(input_shape=(128,128,3),filters=32,kernel_size=(5,5),padding='same',kernel_regularizer=regularizers.l2(0.01),activation='relu'))
model.add(layers.Conv2D(filters=32,kernel_size=(5,5),padding='same',kernel_regularizer=regularizers.l2(0.01),activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(layers.Conv2D(filters=64,kernel_size=(3,3),padding='same',kernel_regularizer=regularizers.l2(0.01),activation='relu'))
#model.add(layers.Conv2D(filters=64,kernel_size=(3,3),padding='same',activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2),strides=(2,2)))

#model.add(layers.Conv2D(filters=256,kernel_size=(3,3),padding='same',activation='relu'))
#model.add(layers.Conv2D(filters=256,kernel_size=(3,3),padding='same',activation='relu'))
#model.add(layers.Conv2D(filters=256,kernel_size=(3,3),padding='same',activation='relu'))
#model.add(layers.MaxPool2D(pool_size=(2,2),strides=(2,2)))

#model.add(layers.Conv2D(filters=512,kernel_size=(3,3),padding='same',activation='relu'))
#model.add(layers.Conv2D(filters=512,kernel_size=(3,3),padding='same',activation='relu'))
#model.add(layers.Conv2D(filters=512,kernel_size=(3,3),padding='same',activation='relu'))
#model.add(layers.MaxPool2D(pool_size=(2,2),strides=(2,2)))

#model.add(layers.Conv2D(filters=512,kernel_size=(3,3),padding='same',activation='relu'))
#model.add(layers.Conv2D(filters=512,kernel_size=(3,3),padding='same',activation='relu'))
#model.add(layers.Conv2D(filters=512,kernel_size=(3,3),padding='same',activation='relu'))
#model.add(layers.MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(layers.Flatten())
model.add(layers.Dense(4096,kernel_regularizer=regularizers.l2(0.01),activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512,kernel_regularizer=regularizers.l2(0.01),activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(12,activation='softmax'))

model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001),
             loss=tf.keras.losses.categorical_crossentropy,
             metrics=['accuracy'])
model.summary()


## Training Model

In [ ]:
train = ImageDataGenerator(validation_split=0.3) 
train_data = train.flow_from_directory(directory='/kaggle/input/plant-seedlings-classification/train',subset='training',target_size=(128,128))
test_data = train.flow_from_directory(directory='/kaggle/input/plant-seedlings-classification/train',subset='validation',target_size=(128,128))

mymodel = ModelCheckpoint("Seedlings Classifier",monitor='val_accuracy',verbose=1,save_best_only=True,save_weights_only=False,mode='auto',period=1) 
stop = EarlyStopping(monitor='val_accuracy',min_delta=0,patience=20,verbose=1,mode='auto')

history = model.fit_generator(steps_per_epoch=100,generator=train_data,validation_data= test_data, validation_steps=10,epochs=50,callbacks=[mymodel,stop])

## Predicting Output

In [ ]:
final_model = tf.keras.models.load_model("Seedlings Classifier")

species = ['Black-grass','Charlock','Cleavers','Common Chickweed','Common wheat','Fat Hen','Loose Silky-bent','Maize','Scentless Mayweed','Shepherds Purse','Small-flowered Cranesbill','Sugar beet']
test = os.listdir('/kaggle/input/plant-seedlings-classification/test')
output = []
for image in test:
    path = "/kaggle/input/plant-seedlings-classification/test/"+image
    img = tf.keras.preprocessing.image.load_img(path,target_size=(128,128))
    img = np.asarray(img)
    img = np.expand_dims(img,axis=0)
    
    prediction = final_model.predict(img)
    prediction = prediction.reshape(-1)
    prediction_list = prediction.tolist()
    max_ind = prediction_list.index(max(prediction_list))
    plant = species[max_ind]
    output.append(plant)

## Creating Final Submission

In [ ]:
submission = pd.DataFrame({'File':test,'Species':output})
submission.to_csv('submission.csv',index=False)